In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

## filepath

In [2]:
#train_data = "../Switchboard-Corpus/swda_data/train_set.txt"
#test_data = "../Switchboard-Corpus/swda_data/test_set.txt"
#val_data = "../Switchboard-Corpus/swda_data/val_set.txt"

## 練習

In [3]:
swda_pass = "../Switchboard-Corpus/swda_data/"
dir_list = ["train", "test", "val"]

In [21]:
len(glob.glob(swda_pass + "test/*"))

19

## 対話ルール設定

In [86]:
user_lists = []

utter_user = []
utter = []
utter_label = []

In [87]:
train_utter_data = []
train_user_data = []
train_label_data = []
test = ""

f = open(swda_pass + "train/2015.txt", "r", encoding='utf-8')
for row in f:
    for i,v in enumerate(row.strip().split("|")):
        
        if i==0 and train_label_data[-1] != v:
            train_label_data.append(v)
    #train_dataset.append(row.strip())
f.close()

In [88]:
utter_user = []
utter = []
utter_label = []

for j in train_dataset:
    for i,v in enumerate(j.split("|")):
        
        if i == 0:
            utter_user.append(v)
        elif i==1:
            utter.append(v)
        else:
            utter_label.append(v)
print("finish")

finish


In [114]:
train_utter_data = []
train_user_data = []
train_label_data = []
text = ""

for i,v in enumerate(utter_user):
    
    if utter_user[i-1] != v:
        #print("実行")
        train_user_data.append(utter_user[i-1])
        train_utter_data.append(text)
        train_label_data.append(utter_label[i-1])
        text = ""
        
    text += utter[i] + " "
    #print(text)

In [115]:
train_utter_data[:10]

["Have you ever gotten one of those calls that is either generated by a computer or somebody going down a list and their either offering a service or they're introducing some new product in the area and normally when they call, you're either in the shower, or you're in the middle of cooking something and you have to stop everything to run to the phone. ",
 "Yes, yes. Is, is that one that you're talking about. ",
 "That was the big one I'm talking about. I work weird hours, and invariably just about the time, I'm going to sleep, the phone tears off the wall. ",
 'Uh-huh, uh-huh. ',
 'And you are trying to crawl out of a half unconscious sleep and answer the phone, you either hear, the as soon as you say hello, you hear the click of the recording coming on, ',
 'Uh-huh. ',
 "or you hear somebody all ready starting, reading off a list of stuff that they've read probably a thousand times that day already. ",
 "That's true, or the ones that are, are generated by a computer. It's just a comp

In [116]:
train_label_data[:10]

['fo_o_fw_"_by_bc', 'qy', 'sd', 'b', 'sd', 'b', 'sd', 'sd', 'sd', 'sd']

In [117]:
train_user_data[:10]

['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B']

In [11]:
labels = set(utter_label)
utter_labels = []
for i in utter_label:
    for j,v in enumerate(labels):
        if i == v:
            utter_labels.append(j)
        

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [12]:
train_data = tf.data.Dataset.from_tensor_slices((utter, utter_labels))

In [23]:
## ボキャブラリーリスト
vocabulary_set = set()
## トークナイザー
tokenizer = tfds.features.text.Tokenizer()

## 分かち書き
for text_tensor,_ in train_data:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)
    
## ボキャブラリーリスト作成
vocab_size = len(vocabulary_set)

In [34]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [63]:
def encode(token, label):
    token = encoder.encode(token.numpy())
    return token, label

@tf.function
def tf_encoder(utter, label):
    encoded_text, label = tf.py_function(encode,[utter, label],[tf.int64, tf.int32])
    encoded_text.set_shape([None])
    label.set_shape([])
    return encoded_text, label

"""
MAX_LENGTH = 10
def filter_max_length(x,y,max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)
"""

'\nMAX_LENGTH = 10\ndef filter_max_length(x,y,max_length=MAX_LENGTH):\n    return tf.logical_and(tf.size(x) <= max_length,\n                        tf.size(y) <= max_length)\n'

In [46]:
train_data

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int32)>

In [54]:
all_train_data_encode = train_data.map(tf_encoder)

In [55]:
all_train_data_encode

<MapDataset shapes: ((None,), ()), types: (tf.int64, tf.int32)>

In [56]:
for i,v in all_train_data_encode.take(5):
    print(i,v)

tf.Tensor([276], shape=(1,), dtype=int64) tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(
[153 238 267 379 300 354 392 196 342 201 367  50 177 196 177 149 196 326
 285 196  60 369 354 336  95 169], shape=(26,), dtype=int64) tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(
[275 305 152 288 114 367 392 288 114 306 305  93 263 405  89  77  93  44
 367 139  71 364 354 201 367 392 368 288 114  94 367 392  24 367 392 226
 196  14 312 196 367 392  14 312 357 259], shape=(46,), dtype=int64) tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor([106], shape=(1,), dtype=int64) tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor([198], shape=(1,), dtype=int64) tf.Tensor(0, shape=(), dtype=int32)


In [57]:
all_train_data_encode = all_train_data_encode.filter(filter_max_length)

In [58]:
for i,v in all_train_data_encode.take(5):
    print(i,v)

tf.Tensor([276], shape=(1,), dtype=int64) tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([106], shape=(1,), dtype=int64) tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor([198], shape=(1,), dtype=int64) tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor([153 387], shape=(2,), dtype=int64) tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor([ 26 228 125 136  77 336  95 169], shape=(8,), dtype=int64) tf.Tensor(6, shape=(), dtype=int32)


In [60]:
BATCH_SIZE = 64
max_len = 40

In [61]:
train_datasets = all_train_data_encode.padded_batch(64, padded_shapes=([max_len], []), drop_remainder=True)

In [75]:
train_batch, train_labels = next(iter(train_datasets))
train_batch.numpy()
train_batch.shape

TensorShape([64, 40])

In [69]:
emb = tf.keras.layers.Embedding(encoder.vocab_size, 64)

In [74]:
result = emb(train_batch)
result.shape

TensorShape([64, 40, 64])

In [76]:
result

<tf.Tensor: shape=(64, 40, 64), dtype=float32, numpy=
array([[[-2.8606737e-02,  2.6519645e-02, -1.4793098e-02, ...,
          4.7928516e-02, -1.8492714e-03,  2.3488626e-03],
        [-3.5874616e-02,  3.6047582e-02, -2.7086401e-02, ...,
          1.5918341e-02,  3.9573919e-02,  4.3381263e-02],
        [-3.5874616e-02,  3.6047582e-02, -2.7086401e-02, ...,
          1.5918341e-02,  3.9573919e-02,  4.3381263e-02],
        ...,
        [-3.5874616e-02,  3.6047582e-02, -2.7086401e-02, ...,
          1.5918341e-02,  3.9573919e-02,  4.3381263e-02],
        [-3.5874616e-02,  3.6047582e-02, -2.7086401e-02, ...,
          1.5918341e-02,  3.9573919e-02,  4.3381263e-02],
        [-3.5874616e-02,  3.6047582e-02, -2.7086401e-02, ...,
          1.5918341e-02,  3.9573919e-02,  4.3381263e-02]],

       [[ 4.3721572e-03,  4.3036234e-02, -4.5510102e-02, ...,
          2.5658082e-02, -4.5279574e-02,  3.8955696e-03],
        [-3.5874616e-02,  3.6047582e-02, -2.7086401e-02, ...,
          1.5918341e-02,  3.9

In [7]:
train_dataset = []
f = open(train_data, "r", encoding='utf-8')
for row in f:
    train_dataset.append(row.strip())
f.close()

In [8]:
len(train_dataset)

192390

In [9]:
test_dataset = []
f = open(test_data, "r", encoding='utf-8')
for row in f:
    test_dataset.append(row.strip())
f.close()

In [10]:
len(test_dataset)

4078

In [11]:
val_dataset = []
f = open(val_data, "r", encoding='utf-8')
for row in f:
    val_dataset.append(row.strip())
f.close()

In [12]:
len(val_dataset)

3272

## データの分割

In [13]:
utter_user = []
utter = []
utter_label = []

In [14]:
train_dataset[0]
train_dataset[0].split("|")

['A', 'Okay.', 'fo_o_fw_"_by_bc']

In [15]:
for j in train_dataset:
    for i,v in enumerate(j.split("|")):
        if i == 0:
            utter_user.append(v)
        elif i==1:
            utter.append(v)
        else:
            utter_label.append(v)
print("finish")

finish


## ウィンドウサイズ

In [16]:
window_size = 5

In [17]:
train_datasets = []
lists = []
for i in range(len(train_dataset)):
    lists.append(train_dataset[i])
    if i%window_size == 4:
        train_datasets.append(lists)
        lists = []

In [18]:
len(train_datasets)

38478

## Tensorflow

In [34]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [42]:
utter_train_dataset = tf.data.Dataset.from_tensor_slices((utter, utter_label))

In [57]:
vocabulary_set = set()
for text_tensor, _ in utter_train_dataset:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)

21852

In [58]:
tokenizer = tfds.features.text.Tokenizer()
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)

In [59]:
encoder.vocab_size

21854

In [62]:
def encode(token, label):
    token = encoder.encode(token.numpy())
    label = np.array([label])
    return token, label

def tf_encoder(utter, label):
    return tf.py_function(encode, [utter, label],[tf.int64, tf.int64])

MAX_LENGTH = 40
def filter_max_length(x,y,max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [63]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [64]:
train_data_set = utter_train_dataset.map(tf_encoder)
train_data_set = train_data_set.filter(filter_max_length)
train_data_set = train_data_set.cache()
train_data_set = train_data_set.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1]))
train_data_set.prefetch(tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: ((None, None), (None, None)), types: (tf.int64, tf.int64)>

In [68]:
def CreateModel(vocab_size, BATCH_SIZE):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, 300, batch_input_shape=[BATCH_SIZE, None]),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform')),
        tf.keras.layers.Dense(vocab_size)
    ])

In [ ]:
model = CreateModel(vocab_size, BATCH_SIZE)

In [5]:
utter_embedding = tf.keras.layers.Embedding(1000, 300)

In [19]:
token = tf.keras.preprocessing.text.Tokenizer()

In [20]:
token.fit_on_texts(utter)
print(token.document_count)

192390


### ベクトル化

In [21]:
sequence_data = token.texts_to_sequences(utter)

In [21]:
## バイナリ表現
matrix_binary = token.sequences_to_matrix(sequence_data, "binary")

In [1]:
## カウント表現
matrix_count = token.sequences_to_matrix(sequence_data, "count")

NameError: name 'token' is not defined

In [22]:
## tfidf
matrix_tfidf = token.sequences_to_matrix(sequence_data, "tfidf")